In [449]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [450]:
# Data Preparation

# Parameters
tickers = ['AAPL', 'COKE', 'F', 'GOOGL', 'IWV']
tickers.sort()

start = '2016-01-01'

# YF download
df = yf.download(tickers, start=start)

# Reformat Dataframe
df = df.stack().reset_index()

# Add daily return column
df['Return'] = df.groupby("Ticker")['Adj Close'].pct_change()

df = df.dropna()

df

[*********************100%%**********************]  5 of 5 completed
C:\Users\andre\AppData\Local\Temp\ipykernel_9612\108948009.py:13: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack().reset_index()


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,Return
5,2016-01-05,AAPL,23.288248,25.677500,26.462500,25.602501,26.437500,223164000,-0.025059
6,2016-01-05,COKE,166.498123,176.740005,179.410004,174.300003,177.020004,32600,0.009655
7,2016-01-05,F,8.630338,13.720000,14.000000,13.510000,13.970000,50267500,-0.017895
8,2016-01-05,GOOGL,38.032848,38.076500,38.459999,37.782501,38.205002,45216000,0.002752
9,2016-01-05,IWV,103.827232,118.669998,118.949997,117.930000,118.690002,314300,0.002450
...,...,...,...,...,...,...,...,...,...
10880,2024-08-27,AAPL,228.091705,228.091705,228.789993,224.889999,226.210007,21174731,0.004013
10881,2024-08-27,COKE,1336.489990,1336.489990,1340.000000,1332.109985,1334.050049,22312,-0.002433
10882,2024-08-27,F,11.175000,11.175000,11.220000,10.990000,11.120000,29799971,0.005851
10883,2024-08-27,GOOGL,165.330002,165.330002,166.442596,164.860001,165.759995,6223392,-0.004995


In [451]:
# Returns dataframe
return_matrix = df[['Date','Ticker','Return']].pivot(index='Date', columns='Ticker', values='Return').dropna()

return_matrix

Ticker,AAPL,COKE,F,GOOGL,IWV
Date,,,,,
2016-01-05,-0.025059,0.009655,-0.017895,0.002752,0.002450
2016-01-06,-0.019570,0.018163,-0.044461,-0.002889,-0.013483
2016-01-07,-0.042205,-0.014004,-0.031274,-0.024140,-0.023917
2016-01-08,0.005288,0.002649,-0.012599,-0.013617,-0.011464
2016-01-11,0.016192,-0.029961,0.018341,0.002955,-0.000797
...,...,...,...,...,...
2024-08-21,-0.000486,0.042052,0.015918,-0.007955,0.003715
2024-08-22,-0.008260,0.026262,0.006452,-0.012361,-0.008657
2024-08-23,0.010288,-0.004417,0.032051,0.011111,0.013320


In [452]:
# Covariance Matrix
cov_matrix = return_matrix.cov().values

cov_matrix

array([[3.33074498e-04, 1.07832030e-04, 1.46702087e-04, 2.07911092e-04,
        1.59794477e-04],
       [1.07832030e-04, 5.69466651e-04, 1.38237996e-04, 9.53204078e-05,
        1.04155245e-04],
       [1.46702087e-04, 1.38237996e-04, 5.54123760e-04, 1.44510455e-04,
        1.64135207e-04],
       [2.07911092e-04, 9.53204078e-05, 1.44510455e-04, 3.20933230e-04,
        1.52565902e-04],
       [1.59794477e-04, 1.04155245e-04, 1.64135207e-04, 1.52565902e-04,
        1.37601582e-04]])

In [453]:
# Create assets dataframe

assets = df.groupby('Ticker')['Return'].mean().rename("Expected Return").to_frame().reset_index()

assets['Beta'] = cov_matrix[-1] / cov_matrix[-1,-1]

assets

,Ticker,Expected Return,Beta
0,AAPL,0.001204,1.161284
1,COKE,0.001249,0.756933
2,F,0.000387,1.192829
3,GOOGL,0.000837,1.108751
4,IWV,0.000587,1.000000


In [454]:
# Expected Returns Vector
expected_returns = assets['Expected Return'].values

expected_returns

array([0.00120399, 0.00124925, 0.00038745, 0.00083743, 0.00058744])

In [455]:
# Beta Vector
betas = assets['Beta'].values

betas

array([1.16128372, 0.75693349, 1.19282936, 1.10875108, 1.        ])

In [456]:
# Portfolio metric functions
def portfolio_xs_return(weights, expected_returns):
    bmk_return = expected_returns[-1]
    return weights @ expected_returns - bmk_return

def portfolio_beta(weights, betas):
    return weights @ betas

def portfolio_tracking_error(current_weights, cov_matrix):
    active_weights = current_weights.copy()
    active_weights[-1] -= 1
    return np.sqrt(active_weights @ cov_matrix @ active_weights)

def negative_information_ratio(current_weights, expected_returns, cov_matrix):
    port_er = portfolio_xs_return(current_weights, expected_returns)
    port_te = portfolio_tracking_error(current_weights, cov_matrix)
    return -port_er / port_te

In [457]:
# Display Weights Function
def display_weights(tickers, weights):
    df = pd.DataFrame()
    df['Ticker'] = tickers
    df['Weight'] = [round(weight, 2) for weight in weights]

    display(df)

In [458]:
# Baseline results

# Initial Weights
size = len(tickers)
initial_weights = np.array([1/size] * size)

# Baseline Results
port_return = portfolio_xs_return(initial_weights, expected_returns)
port_tracking_error = portfolio_tracking_error(initial_weights, cov_matrix)
port_ir = port_return / port_tracking_error

#Display
display_weights(tickers, initial_weights)
print(f"Portfolio Return: {round(port_return,2)} %")
print(f"Portfolio Tracking Error: {round(port_tracking_error,2)} %")
print(f"Portfolio Information Ratio: {round(port_ir,2)}")
print(f"Weights sum to {round(initial_weights.sum(),2)}")

,Ticker,Weight
0,AAPL,0.2
1,COKE,0.2
2,F,0.2
3,GOOGL,0.2
4,IWV,0.2


Portfolio Return: 0.0 %
Portfolio Tracking Error: 0.01 %
Portfolio Information Ratio: 0.04
Weights sum to 1.0


In [459]:
# Optimization

# Constraints and bounds
constraints = [
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # weights sum to 1
    {'type': 'eq', 'fun': lambda x: portfolio_beta(x, betas) - 1},  # beta of 1
    {'type': 'eq', 'fun': lambda x: portfolio_tracking_error(x, cov_matrix) - .05},  # tracking error of .05
  ]

# Result
result = minimize(negative_information_ratio, initial_weights, 
                  args=(expected_returns, cov_matrix),
                  method='SLSQP', constraints=constraints)

optimal_weights = result.x

optimal_weights

array([ 2.71044201,  1.5875263 , -0.5206062 ,  0.45160449, -3.22896661])

In [460]:
# Optimal Results

opt_port_tracking_error = portfolio_tracking_error(optimal_weights,cov_matrix)
opt_port_return = portfolio_xs_return(optimal_weights, expected_returns)
opt_port_beta = portfolio_beta(optimal_weights, betas)
opt_port_information_ratio = opt_port_return / opt_port_tracking_error

# Display
display_weights(tickers,optimal_weights)
print(f"Optimized Portfolio Beta: {round(opt_port_beta,2)}")
print(f"Optimized Portfolio Return: {round(opt_port_return,2) } %")
print(f"Optimized Portfolio Tracking Error: {round(opt_port_tracking_error,2)}")
print(f"Optimized Portfolio Information Ratio: {round(opt_port_information_ratio,2)}")
print(f"Weights sum to: {round(optimal_weights.sum(), 2)}")

,Ticker,Weight
0,AAPL,2.71
1,COKE,1.59
2,F,-0.52
3,GOOGL,0.45
4,IWV,-3.23


Optimized Portfolio Beta: 1.0
Optimized Portfolio Return: 0.0 %
Optimized Portfolio Tracking Error: 0.05
Optimized Portfolio Information Ratio: 0.06
Weights sum to: 1.0
